In [8]:
!pip install "git+https://github.com/facebookresearch/pytorch3d.git"

  Cloning https://github.com/facebookresearch/pytorch3d.git to c:\users\shannon\appdata\local\temp\pip-req-build-m1i5z9we
  Resolved https://github.com/facebookresearch/pytorch3d.git to commit e17ed5cd50a1b43ed60ca4ff7a9a2e329c17c012
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pytorch3d: filename=pytorch3d-0.7.8-cp312-cp312-win_amd64.whl size=874753 sha256=215b4aa713b798bd9de8ccac1ab2950512ca5bf4fa29f48e52d1485961091260
  Stored in directory: C:\Users\shannon\AppData\Local\Temp\pip-ephem-wheel-cache-wz08psfy\wheels\a3\b2\24\3bfb1ad262dd7389a69f1e1aa2afdd7f4f637b3e72dbb9ffa5
Successfully built pytorch3d
  Attempting uninstall: pytorch3d
    Found existing installation: pytorch3d 0.7.8
    Uninstalling pytorch3d-0.7.8:
      Successfully uninstalled pytorch3d-0.7.8


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorch3d.git 'C:\Users\shannon\AppData\Local\Temp\pip-req-build-m1i5z9we'


In [2]:
import sys
import os
import torch as th
import cv2

# set the right device
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# NOTE: assuming we are in `ca_body/notebooks`
sys.path.insert(0, 'data')
from attrdict import AttrDict

from omegaconf import OmegaConf
from torchvision.utils import make_grid

from visualize.ca_body.utils.module_loader import load_from_config
from visualize.ca_body.utils.lbs import LBSModule
from visualize.ca_body.utils.train import load_checkpoint

device = th.device('cuda:0')

获取模型

In [8]:
!curl -O http://audio2photoreal_models.berkeleyvision.org/PXB184_models.tar
!tar -xvf PXB184_models.tar
!del PXB184_models.tar

#REM 下载和解压 RLW104_models.tar
!curl -O http://audio2photoreal_models.berkeleyvision.org/RLW104_models.tar
!tar -xvf RLW104_models.tar
!del RLW104_models.tar

#REM 创建 checkpoints/ca_body/data/ 目录并下载 PXB184.tar.gz
!mkdir checkpoints\ca_body\data
!curl -L -O https://github.com/facebookresearch/ca_body/releases/download/v0.0.1-alpha/PXB184.tar.gz
!tar -xvf PXB184.tar.gz --directory checkpoints\ca_body\data
!del PXB184.tar.gz

#REM 创建 checkpoints/ca_body/data/ 目录并下载 RLW104.tar.gz
!mkdir checkpoints\ca_body\data
!curl -L -O https://github.com/facebookresearch/ca_body/releases/download/v0.0.1-alpha/RLW104.tar.gz
!tar -xvf RLW104.tar.gz --directory checkpoints\ca_body\data
!del RLW104.tar.gz

#REM 下载和解压 asset_models.tar
!curl -O http://audio2photoreal_models.berkeleyvision.org/asset_models.tar
!tar -xvf asset_models.tar
!del asset_models.tar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 1288M    0 13526    0     0  32132      0 11:40:37 --:--:-- 11:40:37 32204
  0 1288M    0 1728k    0     0  1293k      0  0:16:59  0:00:01  0:16:58 1293k
  0 1288M    0 10.4M    0     0  4633k      0  0:04:44  0:00:02  0:04:42 4633k
  1 1288M    1 19.2M    0     0  5942k      0  0:03:41  0:00:03  0:03:38 5944k
  2 1288M    2 28.5M    0     0  6764k      0  0:03:15  0:00:04  0:03:11 6765k
  2 1288M    2 38.2M    0     0  7352k      0  0:02:59  0:00:05  0:02:54 7980k
  3 1288M    3 48.2M    0     0  7851k      0  0:02:48  0:00:06  0:02:42 9619k
  4 1288M    4 58.3M    0     0  8182k      0  0:02:41  0:00:07  0:02:34 9823k
  5 1288M    5 68.5M    0     0  8462k      0  0:02:35  0:00:08  0:02:27  9.9M
  6 1288M    6 78.7M    0     0  8678k      0  0:02

In [11]:
# NOTE: make sure to download the data
model_dir = 'checkpoints/ca_body/data/PXB184/'

ckpt_path = f'{model_dir}/body_dec.ckpt'
config_path = f'{model_dir}/config.yml'
assets_path = f'{model_dir}/static_assets.pt'

# config
config = OmegaConf.load(config_path)
# assets
static_assets = AttrDict(th.load(assets_path))
# sample batch
batch = th.load(f'{model_dir}/sample_batch.pt')
batch = {
    key: val.to(device) if th.is_tensor(val) else val
    for key, val in batch.items()
}
# batch = to_device(batch, device)

C:\Users\shannon\AppData\Local\Temp\ipykernel_25788\3523898037.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  static_assets = AttrDict(th.load(assets_path))
C:\Users\s

In [12]:
batch.keys()

dict_keys(['image', 'ao', 'seg_fg', 'seg_part', 'lbs_motion', 'geom', 'face_embs', 'camera_ids', 'campos', 'camrot', 'focal', 'princpt', 'K', 'Rt', '_index', 'face_R', 'face_t'])

In [13]:
# building the model
model = load_from_config(
    config.model, 
    assets=static_assets,
).to(device)

# loading model checkpoint
load_checkpoint(
    ckpt_path, 
    modules={'model': model},
    # NOTE: this is accounting for difference in LBS impl
    ignore_names={'model': ['lbs_fn.*']},
)

RuntimeError: Not compiled with GPU support
Exception raised from RasterizeMeshesNaive at C:\Users\shannon\AppData\Local\Temp\pip-req-build-m1i5z9we\pytorch3d\csrc\rasterize_meshes/rasterize_meshes.h:138 (most recent call first):
00007FFAA5BA462900007FFAA5BA4580 c10.dll!c10::Error::Error [<unknown file> @ <unknown line number>]
00007FFAA5BA416A00007FFAA5BA4110 c10.dll!c10::detail::torchCheckFail [<unknown file> @ <unknown line number>]
00007FF9125EC66F00007FF9125D58A0 _C.cp312-win_amd64.pyd!c10::ivalue::Object::operator= [<unknown file> @ <unknown line number>]
00007FF9125EC9DE00007FF9125D58A0 _C.cp312-win_amd64.pyd!c10::ivalue::Object::operator= [<unknown file> @ <unknown line number>]
00007FF9125FAB0400007FF9125ECFF0 _C.cp312-win_amd64.pyd!PyInit__C [<unknown file> @ <unknown line number>]
00007FF9125FABA400007FF9125ECFF0 _C.cp312-win_amd64.pyd!PyInit__C [<unknown file> @ <unknown line number>]
00007FF9125E7BA900007FF9125D58A0 _C.cp312-win_amd64.pyd!c10::ivalue::Object::operator= [<u

In [13]:
# disabling training-only stuff
model.learn_blur_enabled = False
model.pixel_cal_enabled = False
model.cal_enabled = False

# forward
with th.no_grad():
    preds = model(**batch)

In [2]:
# visualizing
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
rgb_preds_grid = make_grid(preds['rgb'], nrow=4).permute(1, 2, 0).cpu().numpy() / 255.
plt.figure(figsize=(15, 15))
plt.imshow(rgb_preds_grid[::4,::4])

NameError: name 'preds' is not defined